In [9]:
import requests
from requests import Response
from pydantic import RootModel,BaseModel,Field,field_validator
from datetime import datetime
import pandas as pd 

class Site (BaseModel):
    行政區域:str = Field(alias='sarea')
    總數量:int = Field(alias='total')
    可借數量:int = Field(alias='available_rent_bikes') 
    可還數量:int = Field(alias='available_return_bikes')
    時間:datetime = Field(alias='mday')

    @field_validator('可借數量','可還數量',mode='before')
    @classmethod
    def whitespace_to_zero(cls, value: str) -> str:
        return '0.0' if value == '' else value
    
class Youbike(RootModel):
    root:list[Site]

try:
    youbike_url:Response= requests.get('https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json')
    youbike_url.raise_for_status()
except Exception as e:
    print(e)
else:
    data = Youbike.model_validate_json(youbike_url.text)
    all_sites = data.model_dump()

df = pd.DataFrame(all_sites)
df

,行政區域,總數量,可借數量,可還數量,時間
0,大安區,28,10,18,2024-07-01 19:35:22
1,大安區,21,14,7,2024-07-01 19:22:53
2,大安區,16,9,7,2024-07-01 19:34:19
3,大安區,11,6,5,2024-07-01 19:35:22
4,大安區,16,1,15,2024-07-01 19:33:19
...,...,...,...,...,...
1424,臺大公館校區,30,1,28,2024-07-01 19:35:22
1425,臺大公館校區,20,0,20,2024-07-01 19:31:15
1426,臺大公館校區,24,9,14,2024-07-01 19:29:21
1427,臺大公館校區,40,29,2,2024-07-01 19:34:19


In [14]:
df1 = df.set_index(keys=['行政區域'])
df1.groupby(level='行政區域')[['總數量','可借數量','可還數量']].sum()

,總數量,可借數量,可還數量
行政區域,,,
中山區,3819,1620,2178
中正區,3209,1484,1698
信義區,3331,1339,1904
內湖區,3499,1404,2058
北投區,2472,1004,1440
南港區,2317,1144,1153
士林區,2937,1124,1754
大同區,1629,633,987
大安區,4987,2098,2810
